In [1]:
import os
from collections import Counter
from email.parser import Parser

In [2]:
import codecs
from time import time

from gensim import corpora, models, similarities, logging


In [3]:
rootdir = "C:\\Users\\rohit\\Downloads\\enron_mail_20150507.tar\\enron_mail_20150507\\maildir\\lay-k\\"

In [4]:
def email_analyse(inputfile, to_email_list,from_email_list,email_body):
    with open(inputfile,"r") as f:
        data=f.read()
    
    email = Parser().parsestr(data)
    if email["to"]:
        

        email_to = email['to']
        email_to = email_to.replace("\n","")
        email_to = email_to.replace("\t","")
        email_to = email_to.replace(" ","")

        email_to = email_to.split(",")

        for i in email_to:
            to_email_list.append(i)

        from_email_list.append(email["from"])
        email_body.append(email.get_payload())

In [5]:
to_email_list = list()
from_email_list = list()
email_body = list()

In [6]:
for directory, subdirectory,filenames in os.walk(rootdir):
    for filename in  filenames:
        try:
            email_analyse(os.path.join(directory,filename),to_email_list,from_email_list,email_body)
        except:
            print(os.path.join(directory,filename))

In [7]:
for i in email_body:
    email_body[email_body.index(i)] = i.replace("\n","")

In [8]:
dictionary = corpora.Dictionary(line.lower().split() for line in email_body)

In [9]:
# Filter out the keywords that are only in the corpus once
once_ids = [tokenid for tokenid,
            docfreq in dictionary.dfs.items() if docfreq == 1]

In [10]:
dictionary.filter_tokens(once_ids)

In [11]:
dictionary.compactify()

In [12]:
documents = (line.lower().split() for line in email_body)

In [13]:
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [14]:
# Create TF-IDF

print ("Creating GenSim TF-IDF Model and Index")
tfidfModel = models.TfidfModel(corpus)
tempCorpus = tfidfModel[corpus]

Creating GenSim TF-IDF Model and Index


In [15]:
tfidfIndex = similarities.MatrixSimilarity(tempCorpus)

In [16]:
# Create LSA
print ("Creating GenSim LSA Model and Index")
lsaModel = models.LsiModel(
    corpus, id2word=dictionary, num_topics=300)
tempCorpus = lsaModel[corpus]

lsaIndex = similarities.MatrixSimilarity(tempCorpus)

Creating GenSim LSA Model and Index


In [17]:
# Create LDA
print ("Creating GenSim LDA Model and Index")
ldaModel = models.LdaModel(
    corpus, id2word=dictionary, num_topics=300,
    update_every=1, chunksize=2000, passes=5)
tempCorpus = ldaModel[corpus]
# Create Index
ldaIndex = similarities.MatrixSimilarity(tempCorpus)

Creating GenSim LDA Model and Index
